<a href="https://colab.research.google.com/github/ScientistLim/ProjectAW/blob/feature%2Ftft-model-Hyun/tft_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-forecasting
!pip instsall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 49.1 MB/s eta 0:00:00
ERROR: unknown command "instsall" - maybe you meant "install"


In [3]:
%load_ext tensorboard

In [4]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import datetime  # datetime 모듈 임포트
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import RMSE
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from pytorch_forecasting.models.temporal_fusion_transformer.sub_modules import VariableSelectionNetwork
from torch import nn
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import ModelCheckpoint


In [5]:
# 엑셀 파일 불러오기
file_path = '/content/drive/MyDrive/종설프1_5팀/전처리.ipynb/scaled_data_2m.xlsx'
df = pd.read_excel(file_path)

In [6]:
# 데이터 확인
print(df.head())

                 Time  feed_pressure  scaled_feed_pressure
0 2023-07-09 05:16:00       2.600197              0.144332
1 2023-07-09 05:18:00       2.965855              0.163838
2 2023-07-09 05:20:00       3.331513              0.182233
3 2023-07-09 05:22:00       3.622591              0.199517
4 2023-07-09 05:24:00       3.888415              0.215691


In [10]:
# 데이터 전처리: 필요한 컬럼 설정 (Time, feed_pressure)
df['time'] = pd.to_datetime(df['Time'])
df['time_idx'] = ((df['time'] - df['time'].min()).dt.total_seconds() // 60).astype(int)  # 5분 간격의 시간 인덱스 생성
df['group'] = "feed_pressure"  # 그룹화 컬럼 추가

# 데이터 길이 확인
total_length = len(df)
print(f"Total data length: {total_length} rows")

Total data length: 323843 rows


### 데이터 준비 및 설정
우선 max_encoder_length와 max_prediction_length를 설정하여 데이터의 인코더 및 예측 길이를 정의합니다.

In [11]:
# max_encoder_length와 max_prediction_length 설정
max_encoder_length = 5000  # 30000 - 약 104일 (10분 간격의 데이터)
max_prediction_length = 500  # 3000 - 약 10일 예측

print(f"Max encoder length: {max_encoder_length} entries")
print(f"Max prediction length: {max_prediction_length} entries")

Max encoder length: 5000 entries
Max prediction length: 500 entries


### TimeSeriesDataSet 생성
TimeSeriesDataSet을 사용해 데이터를 생성합니다. 여기에는 타겟, 그룹화 방식, 그리고 변동하는 시계열 컬럼을 정의합니다.


In [12]:
# TimeSeriesDataSet 생성
training = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target="feed_pressure",
    group_ids=["group"],
    max_encoder_length=1000,  # 기존보다 줄인 값
    max_prediction_length=100,  # 기존보다 줄인 값
    min_encoder_length=100,  # 최소 인코더 길이 설정
    time_varying_unknown_reals=["feed_pressure"],
    time_varying_known_reals=["time_idx"],
    target_normalizer=GroupNormalizer(groups=["group"], transformation="softplus"),
    allow_missing_timesteps=True
)

### DataLoader 생성
DataLoader를 생성하여 모델이 데이터를 학습할 수 있도록 준비합니다.

In [13]:
# DataLoader 생성
batch_size = 16
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)

# Validation DataLoader 생성
val_dataloader = training.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

### Temporal Fusion Transformer 모델 생성
Temporal Fusion Transformer 모델을 생성하여 데이터셋을 학습할 준비를 합니다.

In [20]:
# Temporal Fusion Transformer 모델 생성
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.01,
    hidden_size=8,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=RMSE(),
    log_interval=10,
    reduce_on_plateau_patience=3,
)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/models/temporal_fusion_transformer/__init__.py:143: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default optimizer will be 'adam' regardless of whether pytorch_optimizer is installed, in order to minimize the number of 

### LightningModule 정의 및 Trainer 설정
PyTorch Lightning을 이용하여 모델 학습을 보다 쉽게 관리할 수 있도록 합니다.


In [15]:
# LightningModule 정의
class TFTLightningModule(LightningModule):
    def __init__(self, tft_model):
        super().__init__()
        self.tft_model = tft_model.cuda()

    def forward(self, x):
        return self.tft_model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = {key: val.cuda().contiguous() for key, val in x.items()}
        y = y[0].cuda().contiguous() if isinstance(y, tuple) else y.cuda().contiguous()
        y_hat = self(x)
        loss = self.tft_model.loss(y_hat["prediction"], y)
        self.log("train_loss", loss, batch_size=len(x['encoder_cont'].squeeze()))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = {key: val.cuda().contiguous() for key, val in x.items()}
        y = y[0].cuda().contiguous() if isinstance(y, tuple) else y.cuda().contiguous()
        y_hat = self(x)
        loss = self.tft_model.loss(y_hat["prediction"], y)
        self.log("val_loss", loss, batch_size=len(x['encoder_cont'].squeeze()))
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.tft_model.parameters(), lr=0.01, weight_decay=1e-5)
        return optimizer


## 모델 학습

In [22]:
from pytorch_lightning import Trainer

# TFT 모델을 LightningModule로 감싸기
tft_module = TFTLightningModule(tft)

# 학습을 위한 EarlyStopping 설정
early_stop_callback = EarlyStopping(monitor="val_loss", patience=3, min_delta=1e-4)


# 체크포인트 설정
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    filename="tft-best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

# Trainer 설정
trainer = Trainer(
    max_epochs=12,
    accelerator='gpu',
    devices=1,
    gradient_clip_val=0.2,
    callbacks=[early_stop_callback, checkpoint_callback],
    accumulate_grad_batches=2,
    precision=32
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [8]:
# 모델 체크포인트 경로 설정
checkpoint_path = '/content/drive/MyDrive/종설프1_5팀/checkpoints/tft.ckpt'

<span style="color:red;">주의: 체크포인트 삭제!</span>

In [ ]:
# 체크포인트 파일 삭제
if os.path.exists(checkpoint_path):
    os.remove(checkpoint_path)
    print(f"{checkpoint_path} 파일이 삭제되었습니다.")
else:
    print(f"{checkpoint_path} 파일이 존재하지 않습니다.")

In [17]:
# 이전 체크포인트가 있으면 로드
if os.path.exists(checkpoint_path):
    trainer.fit(tft_module, train_dataloader, val_dataloaders=val_dataloader, ckpt_path=checkpoint_path)
    print("이전 체크포인트에서 학습을 이어갑니다.")
else:
    trainer.fit(tft_module, train_dataloader, val_dataloaders=val_dataloader)
    print("새로 학습을 시작합니다.")

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                      | Params | Mode 
----------------------------------------------------------------
0 | tft_model | TemporalFusionTransformer | 5.2 K  | train
----------------------------------------------------------------
5.2 K     Trainable params
0         Non-trainable params
5.2 K     Total params
0.021     Total estimated model params size (MB)
170       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

## 예측

In [19]:
encoder_data = df[df['time_idx'] <= df['time_idx'].min() + 8064]

# 새로운 예측 데이터 생성 (예측 기간 16주)
new_prediction_data = pd.DataFrame({
    "time": pd.date_range(df["time"].max() + pd.Timedelta(minutes=2), periods=80640, freq="2T"),
    "feed_pressure": [0] * 80640,
    "time_idx": range(df["time_idx"].max() + 1, df["time_idx"].max() + 1 + 80640),
    "group": "feed_pressure",
})

# 예측 데이터와 인코더 데이터를 병합
new_data = pd.concat([encoder_data, new_prediction_data]).reset_index(drop=True)

# TimeSeriesDataSet을 예측을 위한 데이터로 생성
new_dataset = TimeSeriesDataSet(
    new_data,
    time_idx="time_idx",
    target="feed_pressure",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_encoder_length=100,
    time_varying_unknown_reals=["feed_pressure"],
    time_varying_known_reals=["time_idx"],
    target_normalizer=GroupNormalizer(groups=["group"], transformation="softplus"),
    allow_missing_timesteps=True,
    predict_mode=True
)

# DataLoader 생성
new_dataloader = new_dataset.to_dataloader(train=False, batch_size=1, num_workers=0)

# 예측 수행
new_predictions = []

# 모델을 CUDA로 이동 (필요한 경우)
tft_module = tft_module.cuda() if torch.cuda.is_available() else tft_module

for batch in new_dataloader:
    # 배치 데이터를 튜플에서 필요한 부분으로 변환 (튜플에서 첫 번째 요소가 데이터로 사용됨)
    if isinstance(batch, tuple):
        x, _ = batch  # 첫 번째 요소인 데이터를 가져옵니다.
    else:
        x = batch

    # 데이터를 CUDA로 이동 (필요할 경우)
    x = {k: v.cuda() for k, v in x.items()} if torch.cuda.is_available() else x

    # 모델 예측
    with torch.no_grad():
        output = tft_module.tft_model(x)
        prediction = output['prediction'].cpu().numpy()

        # 모든 예측 결과를 리스트에 추가
        new_predictions.append(prediction)

<ipython-input-19-530370910ad9>:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  "time": pd.date_range(df["time"].max() + pd.Timedelta(minutes=2), periods=80640, freq="2T"),


NameError: name 'DataLoader' is not defined

In [ ]:
# 예측 결과를 병합
new_predictions = np.concatenate(new_predictions, axis=0)

# 시각화
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.plot(new_prediction_data['time'], new_predictions[:len(new_prediction_data)], label='Predicted Feed Pressure', color='orange')
plt.xlabel('Time')
plt.ylabel('Feed Pressure')
plt.title('16-Week Feed Pressure Prediction')
plt.legend()
plt.grid(True)
plt.show()

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 500 and the array at index 10501 has size 499